# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Cartographie de la ville par le drone ##

On simule une ville avec un petit graphe

In [10]:
import numpy as np

In [11]:
#voici un graph simulant un morceau de ville
#(node1,node2, distance)                                                                                                                                 
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]
print(G)

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]


<img src="assets/img/graph_num.png" width="400">

### 1.2 Calcul du parcours le plus efficace pour le drone ###

L'idée c'est de trouver un chemin eulerien, cependant notre graphe n'est pas eulerien
Nous allons alors utilisé un algo qui transforme un graph non-eulerien en un graph eulerien (sans altéré ses propriétés). Enfin on pourra utiliser un algo qui trouve un chemin eulerien c-a-d un chemin qui passe par toutes les arretes du graphes simulant un parcours de toutes les rues de la ville.

In [12]:
# transformation du graph vers un graph eulerien
def odd_vertices(n, edges):
    deg = [0] * n
    for (a,b,c) in edges:
        deg[a] += 1
        deg[b] += 1
        
    return [a for a in range(n) if deg[a] % 2]

def is_edges(n, edges, node1, node2):
    for (a, b, c) in edges:
        if (a == node1 and b == node2) or (a == node2 and b == node1):
            return (a,b,c)
    return None

def list_edges(n, edges, l_vodd):
    res = []
    for a in l_vodd:
        for b in l_vodd:
            if (a == b):
                continue
            tmp = is_edges(n, edges, a, b)
            if (tmp and tmp not in res) :
                #print("MATCH:", a, b)
                res.append(tmp)
    return res

def shortest_edge_idx(l_edges):
    if (len(l_edges) == 0):
        return None
    shortest = l_edges[0][2]
    shortest_index = 0
    
    index = 1
    for (a,b,c) in l_edges[1:]:
        if (c < shortest):
            shortest = c
            shortest_index = index
        index+=1
    return l_edges[shortest_index]

def transform_to_eulerian(n, edges):
    list_vodd = odd_vertices(n, edges)
    l_edges = list_edges(n, edges, list_vodd)
    
    len_vodd = len(list_vodd)
    if (len_vodd == 2 or len_vodd == 0): #case is already eulerian
        return edges
    #print(list_vodd)
    #print(l_edges)
    while(len(list_vodd) != 2):
        list_vodd = odd_vertices(n, edges)
        l_edges = list_edges(n, edges, list_vodd)
        
        shortest_edge = shortest_edge_idx(l_edges)
        #add edge between two
        (a,b,c) = shortest_edge
        new_edge = (b,a,c)
        edges.append(new_edge)
        
        #delete from vodd
        if (a in list_vodd):
            list_vodd.remove(a)
        if (b in list_vodd):
            list_vodd.remove(b)
        
        #delete from list_edges
        l_edges.remove(shortest_edge)
    return edges

 L'algo de transformation en graph eulerien doit donner le graph ci-dessous.<br>
 Le concept est le suivant doubler certaines des arretes des noeuds impaires pour simuler un graphe Eulerien.<br>
 Ainsi, on pourra utiliser un algo qui trouve un chemin eulerien sur notre graphe.

<img src="assets/img/eulerian_graph.png" width="400">

In [13]:
print("Graph initial:", G)
G_prime = transform_to_eulerian(len(G), G)
print("Graph modifié:", G_prime)

Graph initial: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]
Graph modifié: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]


On a bien une arrête qui s'est rajouté entre les noeuds 6 et 7.<br> Notre algo de transformation vers un graph Eulerien semble fonctionner

In [14]:
from copy import copy

'''
    is_connected - Checks if a graph in the form of a dictionary is 
    connected or not, using Breadth-First Search Algorithm (BFS)
'''
def is_connected(G):
    start_node = list(G)[0]
    color = {v: 'white' for v in G}
    color[start_node] = 'gray'
    S = [start_node]
    while len(S) != 0:
        u = S.pop()
        for v in G[u]:
            if color[v] == 'white':
                color[v] = 'gray'
                S.append(v)
            color[u] = 'black'
    return list(color.values()).count('black') == len(G)

'''
    odd_degree_nodes - returns a list of all G odd degrees nodes
'''
def odd_degree_nodes(G):
    odd_degree_nodes = []
    for u in G:
        if len(G[u]) % 2 != 0:
            odd_degree_nodes.append(u)
    return odd_degree_nodes

'''
    from_dict - return a list of tuples links from a graph G in a 
    dictionary format
'''    
def from_dict(G):
    links = []
    for u in G:
        for v in G[u]:
            links.append((u,v))
    return links

'''
    fleury(G) - return eulerian trail from graph G or a 
    string 'Not Eulerian Graph' if it's not possible to trail a path
'''
def fleury(G):
    '''
        checks if G has eulerian cycle or trail
    '''
    odn = odd_degree_nodes(G)
    #print(odn)
    if len(odn) > 2 or len(odn) == 1:
        return 'Not Eulerian Graph'
    else:
        g = copy(G)
        trail = []
        if len(odn) == 2:
            u = odn[0]
        else:
            u = list(g)[0]
        while len(from_dict(g)) > 0:
            current_vertex = u
            for u in g[current_vertex]:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                bridge = not is_connected(g)
                if bridge:
                    g[current_vertex].append(u)
                    g[u].append(current_vertex)
                else:
                    break
            if bridge:
                g[current_vertex].remove(u)
                g[u].remove(current_vertex)
                g.pop(current_vertex)
            trail.append((current_vertex, u))
    return trail

#adapter le format de notre graph au format dictionnaire de l'algo "fleury"
def to_dict(G):
    dict_graph = {}
    for (a,b) in G:
        dict_graph[a] = []
        dict_graph[b] = []
    for (a,b) in G:
        dict_graph[a].append(b)
        dict_graph[b].append(a)
    return dict_graph



In [19]:
#On utlise toutes les fonctions précedement définies:

def transform_and_find_eulerian_path(graph, verbose=False):
    if (verbose):
        print("Initial Graph:", graph)
    graph = transform_to_eulerian(len(graph), graph)
    graph2 = []
    if (verbose):
        print("To Eulerian Graph:", graph)
    ### on enlève la ponderation
    for (a,b,c) in graph:
        graph2.append((a,b))

    dict_graph2 = to_dict(graph2)
    E_path = fleury(dict_graph2)
    if (verbose):
        print("Eulerian path:", E_path)
    return E_path

In [20]:
path = transform_and_find_eulerian_path(G,True)

Initial Graph: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
To Eulerian Graph: [(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22), (7, 6, 6)]
Eulerian path: [(2, 1), (1, 6), (6, 7), (7, 6), (6, 8), (8, 4), (4, 3), (3, 2), (2, 8), (8, 5)]


Ce qui donne le chemin suivant:
<img src="assets/img/eulerian_path.png" width="400">

In [25]:
# TEST en plus
TEST = [(0,2,12),(2,3,9),(3,0,7),(2,0,23),(2,1,18),(3,1,2),(1,2,4)]
if (transform_and_find_eulerian_path(TEST)):
    print("test: OK")

# TEST encore en plus avec un gros graph
tab = [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1)]
#len(tab)
big_graph_path = transform_and_find_eulerian_path(tab, True)


###TO REMOVE TEST: 
tab = [(1, 2, 25.385), (1, 3, 29.205000000000002), (1, 4, 111.99), (2, 5, 17.679000000000002), (2, 1, 25.385), (2, 6, 159.685), (5, 7, 24.497), (5, 2, 17.679000000000002), (5, 8, 51.457), (7, 9, 29.818), (7, 5, 24.497), (7, 8, 51.29600000000001), (9, 10, 28.581999999999997), (9, 7, 29.818), (10, 11, 14.725000000000001), (10, 9, 28.582), (11, 3, 23.14), (11, 10, 14.725000000000001), (11, 12, 59.021), (6, 2, 159.68500000000003), (6, 4, 46.616), (6, 13, 254.63), (14, 15, 68.87700000000001), (14, 16, 144.09), (14, 17, 75.339), (12, 11, 59.021), (12, 3, 56.389), (3, 1, 29.205), (3, 11, 23.14), (3, 12, 56.388999999999996), (18, 19, 7.494), (18, 20, 12.081), (18, 21, 66.181), (20, 22, 10.047), (20, 18, 12.081), (20, 23, 36.906000000000006), (22, 20, 10.047), (22, 24, 42.646), (22, 25, 14.991), (26, 25, 6.794), (26, 27, 43.654), (26, 28, 15.935), (29, 28, 5.835), (29, 19, 14.132), (29, 30, 71.179), (21, 23, 26.996), (21, 18, 66.181), (21, 31, 77.417), (19, 18, 7.494), (19, 29, 14.132), (19, 30, 77.181), (28, 29, 5.835), (28, 26, 15.935000000000002), (28, 27, 45.327999999999996), (27, 26, 43.654), (27, 28, 45.327999999999996), (27, 32, 416.848), (25, 26, 6.794), (25, 24, 39.528), (25, 22, 14.991), (24, 25, 39.528), (24, 22, 42.646), (24, 33, 616.7019999999999), (31, 34, 41.214999999999996), (31, 21, 77.417), (31, 35, 114.13300000000001), (36, 37, 59.870999999999995), (36, 38, 31.553), (36, 39, 123.821), (38, 40, 77.808), (38, 36, 31.553), (38, 41, 43.632), (42, 43, 49.239000000000004), (42, 40, 28.878999999999998), (42, 44, 31.584999999999997), (45, 46, 5.034), (45, 47, 15.458), (45, 48, 36.0), (47, 45, 15.458000000000002), (47, 49, 3.567), (47, 48, 34.226), (50, 49, 12.619), (50, 51, 5.462000000000001), (50, 52, 15.605999999999998), (51, 53, 19.306), (51, 50, 5.462), (51, 54, 13.79), (55, 54, 2.607), (55, 46, 11.584999999999999), (55, 16, 27.059), (46, 55, 11.585), (46, 45, 5.034), (46, 16, 25.866), (16, 46, 25.866), (16, 55, 27.059), (16, 14, 144.09), (53, 54, 18.481), (53, 51, 19.306), (53, 43, 35.431), (52, 50, 15.605999999999998), (52, 49, 16.006), (52, 56, 38.635000000000005), (49, 47, 3.567), (49, 50, 12.619), (49, 52, 16.006), (48, 47, 34.226), (48, 45, 36.0), (48, 57, 138.80900000000003), (30, 58, 7.244), (30, 19, 77.181), (30, 29, 71.179), (30, 59, 42.777), (60, 61, 309.816), (60, 62, 1222.4200000000003), (60, 63, 272.125), (60, 64, 486.913), (65, 66, 176.056), (65, 67, 299.26), (68, 69, 81.651), (68, 70, 81.546), (33, 71, 651.764), (33, 24, 616.702), (33, 72, 992.9580000000002), (71, 73, 391.8409999999999), (71, 33, 651.7640000000001), (71, 74, 281.125), (75, 76, 16.412), (75, 77, 17.104), (75, 78, 32.661), (79, 80, 14.422), (79, 81, 16.131), (79, 82, 30.774), (80, 83, 14.614), (80, 79, 14.422), (80, 84, 30.935000000000002), (77, 75, 17.104), (77, 85, 15.962), (77, 86, 36.027), (81, 79, 16.131), (81, 76, 10.316), (81, 82, 29.855), (82, 87, 99.638), (82, 79, 30.774), (82, 81, 29.855), (87, 88, 84.878), (87, 82, 99.638), (87, 89, 286.188), (88, 87, 84.878), (88, 90, 107.09), (88, 91, 215.49600000000004), (92, 13, 3.667), (92, 93, 188.751), (92, 94, 72.39099999999999), (94, 95, 259.674), (94, 92, 72.39099999999999), (94, 96, 264.521), (94, 97, 238.968), (96, 94, 264.52099999999996), (95, 98, 590.5000000000001), (95, 94, 259.67400000000004), (95, 99, 864.989), (98, 100, 247.09800000000004), (98, 95, 590.5), (98, 101, 439.841), (102, 103, 104.14900000000002), (102, 100, 151.10999999999999), (102, 104, 103.271), (104, 61, 315.331), (104, 102, 103.271), (104, 103, 156.617), (83, 85, 15.504), (83, 80, 14.614), (83, 84, 31.811999999999998), (99, 95, 864.989), (43, 105, 135.365), (43, 53, 35.431), (43, 42, 49.239000000000004), (43, 106, 102.53), (107, 108, 68.94399999999999), (105, 109, 188.79500000000002), (105, 43, 135.365), (105, 108, 52.83), (110, 108, 32.308), (110, 111, 63.827), (110, 40, 99.896), (108, 110, 32.308), (108, 105, 52.83), (108, 107, 68.944), (112, 113, 51.297), (37, 114, 64.17099999999999), (37, 36, 59.870999999999995), (37, 115, 43.674), (116, 115, 4.943), (115, 116, 4.943), (115, 37, 43.674), (115, 111, 69.757), (117, 111, 38.327), (117, 113, 72.71100000000001), (117, 114, 40.03), (117, 118, 84.74), (111, 117, 38.327), (111, 115, 69.75699999999999), (111, 110, 63.82700000000001), (40, 110, 99.896), (40, 42, 28.878999999999998), (40, 38, 77.80799999999999), (119, 120, 46.96), (119, 121, 101.832), (119, 122, 47.452), (120, 119, 46.96), (120, 123, 50.071000000000005), (120, 124, 157.584), (125, 123, 35.579), (123, 122, 56.212), (123, 125, 35.579), (123, 120, 50.071000000000005), (39, 126, 99.98300000000002), (39, 36, 123.82100000000001), (39, 127, 53.686), (109, 128, 28.8), (109, 105, 188.79500000000004), (109, 121, 30.150999999999996), (118, 117, 84.74000000000001), (118, 129, 50.571), (118, 130, 104.29699999999998), (131, 130, 32.608), (132, 126, 29.896), (126, 132, 29.896), (126, 130, 57.513999999999996), (126, 39, 99.983), (130, 131, 32.608), (130, 118, 104.297), (130, 126, 57.513999999999996), (133, 129, 24.195), (124, 121, 29.877), (124, 120, 157.584), (124, 128, 34.409), (121, 109, 30.150999999999996), (121, 119, 101.832), (121, 124, 29.877), (122, 129, 53.61), (122, 123, 56.212), (122, 119, 47.452), (134, 129, 22.718), (129, 133, 24.195), (129, 134, 22.718), (129, 118, 50.571000000000005), (129, 122, 53.61), (34, 127, 6.099), (34, 31, 41.214999999999996), (34, 135, 122.168), (136, 135, 742.095), (136, 137, 602.008), (136, 138, 731.5280000000001), (137, 136, 602.0079999999999), (137, 139, 321.043), (137, 140, 277.289), (141, 142, 79.502), (141, 143, 169.24299999999997), (141, 127, 170.115), (142, 141, 79.502), (142, 144, 54.635), (142, 72, 224.65699999999998), (142, 145, 123.71900000000001), (143, 146, 56.932), (143, 147, 70.658), (143, 141, 169.243), (127, 34, 6.099), (127, 39, 53.686), (127, 141, 170.11499999999998), (135, 148, 69.543), (135, 34, 122.168), (135, 136, 742.0950000000003), (146, 143, 56.932), (147, 149, 13.809), (147, 23, 39.78999999999999), (147, 143, 70.658), (35, 150, 22.154), (35, 150, 30.871000000000002), (35, 31, 114.13300000000001), (58, 30, 7.244), (58, 150, 86.612), (58, 138, 352.156), (148, 135, 69.543), (145, 151, 54.558), (145, 152, 51.409), (145, 142, 123.719), (153, 72, 216.19199999999998), (153, 154, 1724.0880000000002), (153, 155, 284.17), (155, 153, 284.16999999999996), (155, 56, 512.171), (155, 156, 298.014), (155, 157, 132.365), (158, 159, 7.433), (158, 160, 10.18), (158, 161, 63.839), (162, 163, 25.744999999999997), (162, 164, 11.447), (162, 165, 44.437000000000005), (166, 44, 50.886), (166, 163, 36.856), (166, 167, 13.47), (168, 169, 4.489), (168, 41, 18.271), (168, 170, 36.009), (168, 171, 6.877000000000001), (41, 168, 18.271), (41, 38, 43.632), (41, 169, 19.58), (170, 172, 32.042), (170, 168, 36.009), (170, 173, 150.555), (171, 168, 6.877000000000001), (171, 174, 42.938), (171, 172, 40.138000000000005), (174, 171, 42.938), (174, 172, 12.716000000000001), (174, 167, 30.482), (175, 176, 26.104), (177, 176, 26.747), (177, 178, 129.877), (177, 179, 16.117), (176, 169, 65.977), (176, 177, 26.747), (176, 175, 26.104), (179, 177, 16.117), (173, 170, 150.555), (180, 181, 116.079), (180, 182, 75.041), (180, 183, 73.364), (184, 185, 10.159), (184, 186, 82.226), (184, 187, 40.777), (183, 188, 265.602), (183, 180, 73.364), (183, 157, 36.376000000000005), (189, 190, 105.262), (191, 182, 117.342), (186, 184, 82.226), (192, 193, 120.002), (192, 185, 7.916), (192, 194, 96.379), (182, 191, 117.342), (182, 187, 55.4), (182, 180, 75.041), (181, 180, 116.079), (195, 157, 151.454), (196, 57, 98.961), (193, 192, 120.002), (190, 189, 105.262), (190, 197, 121.901), (190, 198, 56.532), (190, 199, 66.664), (57, 200, 47.661), (57, 48, 138.809), (57, 196, 98.96100000000001), (201, 202, 55.286), (201, 199, 96.081), (201, 203, 180.443), (199, 201, 96.08099999999999), (199, 200, 59.599000000000004), (199, 190, 66.664), (204, 205, 15.563), (204, 194, 201.648), (204, 206, 218.082), (157, 195, 151.454), (157, 183, 36.376000000000005), (157, 155, 132.36499999999998), (207, 208, 353.051), (207, 209, 121.937), (207, 210, 316.02599999999995), (208, 207, 353.051), (211, 210, 173.774), (212, 213, 1486.1060000000002), (212, 214, 874.069), (215, 216, 27.106), (215, 217, 134.552), (215, 218, 88.675), (219, 220, 286.544), (219, 221, 175.85), (219, 222, 1277.8289999999997), (221, 219, 175.85), (221, 223, 727.164), (213, 224, 130.518), (213, 225, 36.283), (213, 212, 1486.1059999999998), (226, 227, 252.76999999999998), (226, 228, 630.0770000000001), (226, 229, 235.87099999999998), (223, 230, 660.826), (223, 221, 727.164), (223, 231, 332.725), (232, 233, 40.446999999999996), (232, 220, 452.13), (232, 234, 700.712), (231, 235, 232.377), (231, 223, 332.725), (227, 236, 352.237), (227, 237, 118.91300000000001), (227, 226, 252.77), (224, 216, 58.136), (224, 213, 130.518), (224, 238, 207.27799999999996), (239, 240, 135.664), (239, 241, 43.06099999999999), (239, 206, 413.619), (239, 214, 416.242), (205, 204, 15.563), (205, 229, 238.567), (205, 242, 454.35200000000003), (234, 232, 700.7119999999999), (234, 67, 573.7189999999999), (234, 243, 147.91500000000002), (241, 244, 4.577), (241, 244, 299.458), (241, 239, 43.061), (245, 244, 49.553), (222, 219, 1277.829), (222, 246, 624.568), (247, 206, 211.412), (240, 239, 135.664), (244, 241, 4.577), (244, 241, 299.45799999999997), (244, 245, 49.553), (229, 226, 235.87100000000004), (229, 205, 238.567), (229, 248, 274.974), (206, 247, 211.412), (206, 204, 218.082), (206, 239, 413.619), (248, 237, 253.48799999999997), (248, 249, 176.467), (248, 229, 274.974), (246, 222, 624.568), (217, 215, 134.552), (250, 230, 170.53399999999996), (251, 220, 175.652), (220, 232, 452.13), (220, 219, 286.54400000000004), (220, 251, 175.65200000000002), (252, 225, 112.374), (252, 78, 687.733), (252, 233, 112.90299999999999), (90, 253, 16.117), (90, 243, 465.401), (90, 88, 107.09), (254, 255, 102.22), (216, 215, 27.106), (216, 224, 58.136), (216, 256, 169.52), (218, 257, 79.86099999999999), (218, 215, 88.675), (218, 258, 141.223), (259, 253, 53.705999999999996), (67, 234, 573.7189999999999), (67, 65, 299.26), (67, 91, 674.6709999999999), (260, 261, 68.97399999999999), (260, 255, 33.576), (260, 253, 110.421), (258, 218, 141.223), (256, 216, 169.52), (243, 234, 147.915), (243, 90, 465.40100000000007), (243, 262, 106.924), (261, 260, 68.97399999999999), (253, 90, 16.117), (253, 259, 53.705999999999996), (253, 260, 110.42099999999999), (255, 260, 33.576), (255, 91, 67.867), (255, 254, 102.22), (262, 243, 106.924), (91, 255, 67.867), (91, 67, 674.6709999999999), (91, 88, 215.496), (263, 233, 95.514), (263, 225, 95.355), (264, 230, 143.26999999999998), (230, 250, 170.534), (230, 223, 660.826), (230, 264, 143.27), (214, 265, 276.84499999999997), (214, 239, 416.242), (214, 212, 874.0690000000001), (265, 214, 276.84499999999997), (266, 156, 854.078), (266, 267, 541.171), (266, 268, 152.649), (185, 192, 7.916), (185, 184, 10.159), (268, 266, 152.649), (194, 204, 201.64800000000002), (194, 192, 96.379), (225, 233, 6.683), (225, 263, 95.355), (225, 252, 112.374), (225, 213, 36.283), (269, 270, 478.15200000000004), (269, 271, 1264.4689999999998), (269, 272, 581.6820000000001), (273, 274, 217.434), (275, 276, 249.902), (275, 277, 554.3979999999999), (275, 278, 576.617), (275, 279, 981.047), (280, 274, 359.55899999999997), (280, 281, 61.991), (280, 282, 67.512), (272, 269, 581.682), (276, 283, 970.4069999999999), (276, 271, 299.38), (276, 275, 249.902), (276, 284, 836.1009999999998), (274, 285, 713.2800000000001), (274, 280, 359.55899999999997), (274, 273, 217.43400000000003), (271, 286, 151.52100000000002), (271, 269, 1264.469), (271, 276, 299.37999999999994), (285, 274, 713.28), (285, 270, 348.211), (285, 287, 446.03700000000003), (270, 285, 348.211), (270, 269, 478.152), (286, 271, 151.52100000000002), (281, 288, 96.653), (281, 103, 259.837), (281, 280, 61.991), (281, 289, 904.5020000000001), (290, 288, 155.55), (291, 93, 196.096), (288, 97, 177.679), (288, 281, 96.653), (288, 290, 155.55), (103, 281, 259.83700000000005), (103, 102, 104.149), (103, 104, 156.617), (100, 102, 151.11), (100, 98, 247.09799999999998), (100, 292, 433.379), (97, 93, 30.434), (97, 288, 177.679), (97, 94, 238.96799999999996), (293, 4, 114.84700000000001), (93, 92, 188.751), (93, 97, 30.434), (93, 291, 196.096), (292, 100, 433.37899999999996), (294, 64, 271.145), (64, 295, 369.788), (64, 294, 271.145), (64, 296, 489.38199999999995), (64, 60, 486.913), (297, 298, 35.972), (297, 299, 100.181), (297, 300, 42.559), (301, 62, 546.9879999999999), (301, 300, 251.213), (301, 302, 303.18), (300, 301, 251.213), (300, 298, 23.426000000000002), (300, 297, 42.559), (303, 101, 230.068), (298, 297, 35.972), (298, 300, 23.426000000000002), (298, 304, 549.7919999999999), (101, 98, 439.84099999999995), (101, 303, 230.06799999999998), (101, 305, 479.211), (302, 301, 303.17999999999995), (277, 279, 980.903), (277, 275, 554.3980000000001), (277, 139, 33.801), (139, 304, 164.00900000000001), (139, 277, 33.801), (139, 137, 321.043), (304, 298, 549.792), (304, 139, 164.009), (304, 306, 1495.766), (296, 64, 489.38200000000006), (307, 295, 656.345), (295, 64, 369.788), (295, 307, 656.3449999999999), (308, 309, 380.41400000000004), (310, 309, 180.9), (310, 311, 383.75), (312, 313, 175.81900000000002), (314, 315, 451.792), (314, 316, 389.913), (314, 317, 357.068), (318, 138, 1.417), (318, 319, 478.885), (318, 32, 237.763), (320, 321, 603.901), (316, 313, 585.211), (316, 322, 686.908), (316, 321, 707.934), (316, 314, 389.9129999999999), (322, 319, 3.916), (322, 316, 686.908), (322, 323, 454.831), (140, 319, 915.3039999999999), (140, 137, 277.289), (140, 321, 251.32599999999996), (313, 309, 532.2980000000001), (313, 316, 585.2109999999999), (313, 312, 175.81900000000002), (324, 323, 178.528), (324, 325, 177.051), (324, 326, 199.021), (315, 314, 451.79200000000003), (323, 322, 454.83099999999996), (323, 317, 421.87300000000005), (323, 324, 178.52800000000002), (321, 320, 603.901), (321, 140, 251.32600000000002), (321, 316, 707.934), (319, 322, 3.916), (319, 318, 478.88499999999993), (319, 140, 915.304), (309, 310, 180.9), (309, 313, 532.298), (309, 308, 380.414), (138, 318, 1.417), (138, 58, 352.156), (138, 136, 731.528), (32, 318, 237.763), (32, 27, 416.848), (32, 325, 460.927), (32, 327, 1840.7620000000006), (279, 275, 981.047), (279, 328, 283.348), (279, 277, 980.9030000000001), (327, 329, 987.878), (327, 32, 1840.7620000000006), (330, 331, 160.64800000000002), (331, 330, 160.64799999999997), (331, 326, 990.1630000000001), (331, 332, 68.09700000000001), (8, 7, 51.29600000000001), (8, 5, 51.45700000000001), (333, 334, 375.919), (333, 335, 313.551), (333, 236, 170.059), (237, 227, 118.91299999999998), (237, 248, 253.488), (237, 334, 72.418), (336, 283, 545.064), (336, 334, 81.709), (336, 337, 21.457), (334, 336, 81.709), (334, 237, 72.41799999999999), (334, 333, 375.919), (338, 339, 132.218), (339, 89, 84.51899999999999), (339, 338, 132.218), (340, 284, 313.58099999999996), (340, 283, 1024.4140000000002), (340, 69, 20.608), (236, 86, 433.52099999999996), (236, 227, 352.237), (236, 333, 170.05900000000003), (283, 276, 970.4069999999998), (283, 336, 545.064), (283, 340, 1024.4140000000002), (341, 342, 7.075), (341, 343, 626.7819999999997), (344, 345, 496.7210000000001), (326, 331, 990.1629999999998), (326, 324, 199.021), (326, 346, 506.39500000000004), (345, 347, 90.034), (345, 344, 496.721), (345, 346, 226.921), (348, 311, 48.277), (348, 349, 455.18499999999995), (348, 350, 266.095), (346, 317, 727.214), (346, 326, 506.395), (346, 345, 226.921), (350, 348, 266.09499999999997), (317, 314, 357.068), (317, 323, 421.873), (317, 346, 727.214), (351, 69, 70.312), (351, 70, 69.941), (351, 84, 1290.3850000000002), (329, 73, 164.579), (329, 327, 987.878), (329, 352, 300.20799999999997), (353, 235, 770.9920000000001), (66, 65, 176.05599999999998), (343, 341, 626.7819999999998), (343, 347, 445.49399999999997), (342, 341, 7.075), (70, 69, 5.982), (70, 68, 81.546), (70, 351, 69.941), (70, 354, 410.48300000000006), (354, 70, 410.48299999999995), (287, 285, 446.03700000000003), (287, 289, 21.032999999999998), (287, 355, 18.39), (355, 356, 4.927), (355, 287, 18.39), (355, 357, 45.06), (356, 355, 4.927), (356, 63, 12.662), (356, 357, 48.42399999999999), (289, 281, 904.502), (289, 63, 16.762999999999998), (289, 287, 21.032999999999998), (63, 356, 12.661999999999999), (63, 289, 16.762999999999998), (63, 60, 272.125), (54, 53, 18.481), (54, 51, 13.79), (54, 55, 2.607), (332, 358, 34.816), (332, 359, 43.897000000000006), (332, 331, 68.09700000000001), (358, 332, 34.816), (359, 332, 43.897000000000006), (154, 153, 1724.0879999999997), (154, 360, 302.442), (154, 73, 691.4770000000001), (361, 362, 3.917), (362, 363, 12.726), (362, 361, 3.917), (362, 267, 27.000999999999998), (363, 364, 5.832), (363, 362, 12.725999999999999), (363, 267, 28.702), (364, 363, 5.832), (364, 365, 24.439999999999998), (267, 266, 541.171), (267, 362, 27.000999999999998), (267, 363, 28.702), (365, 366, 131.349), (365, 364, 24.439999999999998), (367, 156, 1426.1480000000001), (367, 366, 428.94999999999993), (367, 368, 168.85299999999998), (156, 266, 854.078), (156, 155, 298.01399999999995), (156, 367, 1426.148), (73, 71, 391.841), (73, 329, 164.579), (73, 154, 691.4770000000001), (233, 225, 6.683), (233, 263, 95.514), (233, 232, 40.447), (233, 252, 112.90299999999999), (78, 76, 29.223999999999997), (78, 75, 32.661), (78, 252, 687.7330000000001), (76, 81, 10.316), (76, 75, 16.412000000000003), (76, 78, 29.223999999999997), (85, 77, 15.962), (85, 83, 15.504000000000001), (85, 86, 34.215), (84, 83, 31.811999999999998), (84, 80, 30.935000000000002), (84, 351, 1290.385), (86, 236, 433.52099999999996), (86, 77, 36.027), (86, 85, 34.215), (69, 351, 70.312), (69, 68, 81.651), (69, 70, 5.982), (69, 340, 20.608), (284, 276, 836.1009999999999), (284, 340, 313.581), (360, 369, 67.46300000000001), (360, 154, 302.442), (74, 71, 281.125), (370, 371, 216.294), (371, 278, 949.758), (371, 370, 216.294), (371, 357, 718.7479999999999), (325, 372, 307.707), (325, 32, 460.927), (325, 324, 177.051), (372, 325, 307.707), (257, 228, 92.599), (257, 218, 79.86099999999999), (257, 373, 101.914), (373, 257, 101.91399999999999), (228, 374, 81.888), (228, 226, 630.077), (228, 257, 92.599), (374, 228, 81.888), (375, 128, 208.09300000000002), (59, 30, 42.777), (149, 147, 13.809), (376, 152, 36.242999999999995), (151, 145, 54.558), (152, 377, 95.71000000000001), (152, 145, 51.409), (152, 376, 36.242999999999995), (377, 378, 21.004), (377, 379, 58.183), (377, 152, 95.71000000000001), (378, 377, 21.004), (379, 377, 58.183), (278, 371, 949.758), (278, 380, 68.02099999999999), (278, 275, 576.617), (380, 278, 68.021), (299, 297, 100.181), (61, 60, 309.81600000000003), (61, 104, 315.331), (61, 381, 132.596), (381, 61, 132.596), (17, 128, 114.09599999999999), (17, 328, 48.486000000000004), (17, 14, 75.339), (17, 15, 11.971), (328, 279, 283.348), (328, 15, 51.866), (328, 17, 48.486000000000004), (357, 356, 48.42400000000001), (357, 355, 45.06), (357, 371, 718.748), (4, 6, 46.616), (4, 1, 111.99), (4, 293, 114.84700000000001), (23, 21, 26.996), (23, 20, 36.906), (23, 147, 39.79), (106, 43, 102.53), (382, 44, 23.025), (382, 382, 74.994), (382, 382, 74.994), (44, 42, 31.584999999999997), (44, 166, 50.885999999999996), (44, 382, 23.025), (44, 383, 31.999), (172, 174, 12.716), (172, 170, 32.042), (172, 171, 40.138000000000005), (352, 329, 300.208), (384, 385, 31.076), (385, 386, 26.126), (385, 384, 31.076), (385, 387, 48.896), (388, 386, 30.443), (387, 389, 15.354), (387, 144, 32.553), (387, 385, 48.896), (389, 387, 15.354), (386, 385, 26.126), (386, 388, 30.443), (386, 144, 49.001), (144, 142, 54.635), (144, 387, 32.553), (144, 386, 49.001), (390, 238, 313.712), (187, 391, 129.513), (187, 184, 40.777), (187, 182, 55.4), (391, 187, 129.513), (203, 201, 180.44299999999998), (203, 188, 142.25900000000001), (203, 197, 87.404), (392, 393, 32.549), (393, 198, 38.581), (393, 392, 32.549), (393, 394, 84.89999999999999), (394, 393, 84.89999999999999), (395, 210, 102.567), (210, 207, 316.02599999999995), (210, 211, 173.774), (210, 395, 102.567), (235, 353, 770.992), (235, 231, 232.377), (347, 345, 90.034), (347, 343, 445.49399999999997), (347, 396, 75.64500000000001), (396, 347, 75.64500000000001), (161, 397, 45.662), (161, 164, 62.16), (161, 158, 63.83899999999999), (397, 161, 45.662), (128, 109, 28.8), (128, 375, 208.093), (128, 17, 114.09599999999999), (128, 124, 34.409), (398, 306, 841.0059999999997), (398, 399, 51.19499999999999), (399, 398, 51.19499999999999), (399, 400, 18.038), (305, 101, 479.211), (401, 311, 297.801), (311, 348, 48.277), (311, 310, 383.75), (311, 401, 297.801), (349, 348, 455.18499999999995), (306, 304, 1495.7659999999998), (306, 398, 841.006), (402, 335, 182.27100000000002), (403, 238, 319.122), (238, 224, 207.278), (238, 390, 313.712), (238, 403, 319.122), (335, 402, 182.271), (335, 333, 313.55100000000004), (335, 404, 187.73499999999996), (404, 335, 187.735), (89, 339, 84.51899999999999), (89, 87, 286.188), (369, 368, 17.641), (369, 360, 67.463), (368, 367, 168.853), (368, 369, 17.641000000000002), (366, 365, 131.349), (366, 367, 428.95), (15, 328, 51.866), (15, 14, 68.87700000000001), (15, 17, 11.971), (405, 62, 165.982), (62, 60, 1222.4199999999998), (62, 301, 546.988), (62, 405, 165.98199999999997), (163, 406, 15.928), (163, 166, 36.855999999999995), (163, 162, 25.744999999999997), (169, 168, 4.489), (169, 176, 65.97699999999999), (169, 41, 19.58), (72, 142, 224.657), (72, 153, 216.192), (72, 33, 992.9579999999999), (209, 207, 121.937), (209, 202, 17.972), (209, 249, 21.715), (202, 201, 55.286), (202, 242, 14.042), (202, 209, 17.972), (242, 205, 454.352), (242, 249, 14.268999999999998), (242, 202, 14.042000000000002), (249, 248, 176.46699999999998), (249, 209, 21.715), (249, 242, 14.268999999999998), (337, 336, 21.457), (400, 399, 18.038), (13, 92, 3.667), (13, 6, 254.63), (13, 407, 57.723), (407, 13, 57.723), (282, 280, 67.512), (178, 177, 129.877), (159, 158, 7.433), (160, 158, 10.18), (383, 44, 31.999000000000002), (408, 113, 37.198), (113, 112, 51.297), (113, 117, 72.71100000000001), (113, 408, 37.198), (409, 200, 61.745), (200, 199, 59.59899999999999), (200, 57, 47.661), (200, 409, 61.745000000000005), (410, 188, 30.838), (188, 203, 142.259), (188, 183, 265.602), (188, 410, 30.838), (198, 393, 38.581), (198, 197, 71.188), (198, 190, 56.532), (197, 198, 71.188), (197, 190, 121.901), (197, 203, 87.404), (150, 58, 86.612), (150, 35, 22.154), (150, 35, 30.871000000000002), (165, 162, 44.437), (411, 164, 67.815), (164, 162, 11.447), (164, 161, 62.16), (164, 411, 67.815), (412, 413, 17.893), (412, 414, 17.582), (412, 415, 37.626000000000005), (414, 415, 16.643), (414, 416, 3.057), (414, 412, 17.582), (417, 418, 22.35), (417, 415, 33.302), (417, 167, 17.176000000000002), (415, 414, 16.643), (415, 417, 33.302), (415, 412, 37.626000000000005), (413, 412, 17.893), (413, 419, 14.254), (413, 420, 28.799), (421, 419, 19.301), (421, 418, 16.99), (421, 420, 35.786), (420, 419, 20.816), (420, 413, 28.799), (420, 421, 35.786), (419, 413, 14.254), (419, 421, 19.301), (419, 420, 20.816), (419, 416, 16.618), (416, 414, 3.057), (416, 418, 21.025), (416, 419, 16.618), (418, 416, 21.025), (418, 417, 22.35), (418, 421, 16.99), (167, 174, 30.482), (167, 166, 13.47), (167, 417, 17.176000000000002), (406, 163, 15.928), (422, 56, 39.706), (56, 422, 39.706), (56, 52, 38.635000000000005), (56, 155, 512.171), (114, 117, 40.03), (114, 37, 64.17099999999999), (114, 423, 42.54), (423, 114, 42.54)]
big_graph_path = transform_and_find_eulerian_path(tab, True)

test: OK
Initial Graph: [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1)]
To Eulerian Graph: [(0, 1, 9), (1, 2, 28), (1, 22, 27), (2, 3, 6), (3, 4, 13), (3, 6, 20), (4, 5, 25), (5, 6, 11), (5, 14, 18), (6, 7, 7), (7, 8, 18), (8, 9, 13), (8, 10, 29), (10, 11, 9), (11, 12, 10), (11, 18, 7), (12, 13, 11), (12, 14, 2), (13, 14, 5), (13, 15, 5), (15, 16, 11), (15, 21, 11), (16, 17, 18), (16, 21, 19), (17, 18, 23), (17, 19, 4), (18, 19, 24), (19, 20, 9), (21, 22, 1), (14, 12, 2), (19, 17, 4), (15, 13, 5), (18, 11, 7), (1, 0, 9), (6, 5, 11), (9, 8, 13), (21, 16, 19)]
Eulerian path: [(3, 2), (2, 1), (1, 0), (0, 1), (1, 22), (22, 21), (21, 15), (15, 13), (13, 12), (12, 11), (11

Eulerian path: [(1, 2), (2, 5), (5, 7), (7, 9), (9, 10), (10, 11), (11, 3), (3, 1), (1, 4), (4, 6), (6, 2), (2, 1), (1, 3), (3, 12), (12, 11), (11, 3), (3, 12), (12, 11), (11, 10), (10, 9), (9, 7), (7, 5), (5, 8), (8, 7), (7, 8), (8, 5), (5, 2), (2, 6), (6, 13), (13, 92), (92, 93), (93, 291), (291, 93), (93, 97), (97, 94), (94, 92), (92, 94), (94, 95), (95, 98), (98, 100), (100, 102), (102, 103), (103, 104), (104, 102), (102, 104), (104, 61), (61, 60), (60, 62), (62, 301), (301, 300), (300, 297), (297, 298), (298, 300), (300, 301), (301, 302), (302, 301), (301, 62), (62, 405), (405, 62), (62, 60), (60, 63), (63, 356), (356, 355), (355, 287), (287, 285), (285, 274), (274, 273), (273, 274), (274, 280), (280, 281), (281, 288), (288, 290), (290, 288), (288, 97), (97, 288), (288, 281), (281, 103), (103, 281), (281, 280), (280, 282), (282, 280), (280, 274), (274, 285), (285, 270), (270, 269), (269, 271), (271, 276), (276, 275), (275, 277), (277, 279), (279, 275), (275, 278), (278, 371), (371

Voici un test sur un graph un peu plus gros
<img src="assets/img/big_path.png" width="500">

On obtient peut-être pas le chemin "le" plus optimisé mais en tout cas on a la garantie d'avoir un chemin efficace.

### 1.3 Recuperation des données du drones (simulation d'eneigement) ###

In [297]:
#ICI on simule l'eneigement (baill de plustard)